In [1]:
from collections import defaultdict
from importlib import reload
from itertools import product
from pathlib import Path
from time import perf_counter

import sys
sys.path.append("../src")
import convex
import frank_wolfe
import main_gaussian
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm



[KeOps] Warning : There were warnings or errors compiling formula :
<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.

[KeOps] Warning : 
    The location of Cuda header files cuda.h and nvrtc.h could not be detected on your system.
    You must determine their location and then define the environment variable CUDA_PATH,
    either before launching Python or using os.environ before importing keops. For example
    if these files are in /vol/cuda/10.2.89-cudnn7.6.4.38/include you can do :
      import os
      os.environ['CUDA_PATH'] = '/vol/cuda/10.2.89-cudnn7.6.4.38'
    
[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors compiling formula :
/home/luchar/.pyenv/versions/3.11.4/lib/python3.11/site-packages/keopscore/binders/nvrtc/nvrtc_jit.cpp:5:10: fatal error: nvrtc.h: No such file or directory
    5 | #include <nvrtc.h>
      |          ^~~~~~~~~
compilation terminated.

OK
[pyKeOps] Compiling nvrtc bin

In [327]:
data = utils.get_data(
    dataset="gaussian",
    num_buyer=30,
    num_seller=1000,
    num_val=5,
    dim=10,
    noise_level=0.1,
)

x_s = data["X_sell"].astype(np.single)
y_s = data["y_sell"]
x_val = data["X_val"].astype(np.single)
y_val = data["y_val"]
x_b = data["X_buy"].astype(np.single)
coef = data.get("coef")
y_b = (data["X_buy"] @ coef) if coef is not None else data["y_buy"]

In [ ]:
rang = list(range(2, 20, 1)) + list(range(20, 150, 5))

errors = defaultdict(list)
runtimes = defaultdict(list)

for j in range(0, x_b.shape[0]):
    x_test = x_b[j : j + 1]
    y_test = y_b[j : j + 1]

    res_fw = main_gaussian.design_selection(
        (x_s, y_s),
        (x_test, y_test),
        num_select=10,
        num_iters=500,
        alpha=0.1,
        recompute_interval=0,
        line_search=True,
    )
    w_fw = res_fw["weights"]
    w_os = frank_wolfe.one_step(x_s, x_test)
    errors['Ours (multi-step)'].append([utils.get_error(x_test, y_test, x_s, y_s, w_fw, k) for k in rang])
    errors['Ours (single step)'].append([utils.get_error(x_test, y_test, x_s, y_s, w_os, k) for k in rang])


    w_baselines = main_gaussian.get_baseline_values(
        x_s, y_s, x_val, y_val, np.ones_like(x_val), np.ones_like(y_val), 
        baselines=[
            'DataShapley',
            # 'DataBanzhaf',
            # 'BetaShapley',
            'LeaveOneOut',
            "DataOob",
            'KNNShapley',
            'LavaEvaluator',
            # 'RandomEvaluator',
            'DVRL',
            'InfluenceSubsample',
        ],
        baseline_kwargs={
            'DataShapley': {'mc_epochs': 100, 'models_per_iteration': 10},
            # 'DataBanzhaf': {'mc_epochs': 100, 'models_per_iteration': 10},
            # 'BetaShapley': {'mc_epochs': 100, 'models_per_iteration': 10},
            'DataOob': {'num_models': 100},
            'KNNShapley': {},
            'LavaEvaluator': {},
            'DVRL': {'rl_epochs': 100},
            'InfluenceSubsample': {'num_models': 100},
            'LeaveOneOut': {},
        },

    )
    values, times = w_baselines 

    for k, v in values.items():
        errors[k].append([utils.get_error(x_test, y_test, x_s, y_s, v, k) for k in rang])

    w_rand = np.random.permutation(len(x_s))
    errors['Random'].append([utils.get_error(x_test, y_test, x_s, y_s, w_rand, k) for k in rang])
    
    for k, v in times.items():
        runtimes[k].append(v)
    
    print(f'round {j} done')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 566.38it/s]


--------------DataShapley---------------
Start: marginal contribution computation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:15<00:00,  7.55s/it]


gr_stat=100


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:15<00:00,  7.58s/it]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:40<00:00, 10.09s/it]


gr_stat=100


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:04<00:00, 12.47s/it]


gr_stat=100


  0%|                                                                                                                                                          | 0/10 [00:00<?, ?it/s]

In [ ]:
figure_dir = Path('../figures')

In [ ]:
plt.rcParams["font.family"] = "serif"
plt.figure(figsize=(8, 8))
for k, v in errors.items():
    err = np.array(v).mean(0).squeeze()
    ms = 10
    match k:
        case 'LavaEvaluator':
            k = 'LAVA'
        case 'InfluenceSubsample':
            k = 'Influence'
        case 'LeaveOneOut':
            k = 'Leave One Out'
        case 'KNNShapley':
            k = 'KNN Shapley'
        case 'DataOob':
            k = 'Data-OOB'
    match k:
        case k if 'Ours' in k:
            lw = 2
            ls = '-'
            marker = '*'
            ms = ms + 5
        case k if 'random' in k.lower():
            lw = 5
            ls = '-'
            marker = ''
        case _:
            lw = 2
            ls = '-'
            marker = 's'
    plt.plot(rang, err, label=k, marker=marker, ls=ls, lw=lw, ms=ms)
plt.xticks(np.arange(2, 21, 2), fontsize='x-large')
plt.yticks(np.arange(0, 10, 1.0), fontsize='x-large')
# plt.xlim(1.5, 21)
# plt.ylim(0, 2.8)
    
plt.xlim(1.5, 10)
plt.ylim(0, 4)
# plt.ylim(0, 1.3)
# plt.ylim(0, 0.3)
plt.xlabel('Number of Datapoints selected', fontsize='xx-large', labelpad=8)
plt.ylabel('Test\nError', fontsize='xx-large', rotation=0, labelpad=40)
plt.legend(fontsize='xx-large', bbox_to_anchor=(0.5, 1.4), loc='upper center', ncols=2)
# plt.legend(fontsize='x-large', loc='upper right')
plt.tight_layout(pad=0, w_pad=0)
plt.savefig(figure_dir / f"overfitting-low-val.eps", bbox_inches="tight")